# Overview

The Goal of this exercise is to learn how to integrate and deploy the models you build using Pytorch, in to a real world application. Here are the steps we follow to deploy our application
* Build the required web application UI using python `Flask` micro framework.
* Use the existing pre-built Pytorch `densnet121` model and integrate it with our web application
* Learn multiple ways in which we can deploy the application in Azure cloud infrastructure.

The `densnet121` model was originally built to identify 1000 different classes. We keep the original model as is to predict all the image classes. Here is a quick peek at how the end application looks like. 

![](images/preview_ostrich.png "Web app preview")

you can have a look at the deployed app [here](http://52.157.86.185:8000) (**TODO: change the link**)
<br>
Here are the steps we will follow:

* Build the `html` pages required for UI.
* Make the `html` pages interactive by creating `routes`.
* Build the model.
* Deploy the web app using following options:
    * Virtual Machines in Azure environment.
    * "Azure app services" mechanism using code present in github.
    * Build a docker image and deploy the docker image using "Azure app services".

### Initial build setup.

Let us have a separate virtual environment for our development.

```sh
$ sudo apt install python3-venv
$ python3 -m venv webappenv
$ source ./webapp/bin/activate
```

Here is the github repo which we will refer to keep track of our progress and reference.

```sh
$ git clone https://github.com/bigvisionai/pytorch-web-app-deploy-azure
$ cd pytorch-web-app-deploy-azure
$ pip install -r requirements.txt
```

Above steps should install all the dependencies. We are all set to build the Web app!


## Building  UI:

It is good to have a general mental model about our final application. No, we are not talking about the aesthetic of the application. We are talking about, how the user is going to interact with our application. We know that we will be working with images, so we have two options, 
<br>
* Use the images present in the user's local disk.
* Use the images present in the internet.
* Use the images present in some private or public cloud, such as Azure data blob or AWS S3 buckets etc.

<br>
The possibilities are many. These kind of thoughts will prompt us to look for the next steps in building our application.
<br>
To keep things simiple, in this excercise we will go with user uploading images from his local disk. Once you get the hang of the process, we encourage you to build or extend this application to pick images from multiple sources.
<br>
So we need to provide a button option to the end user to click and upload the image. Once user selects the image, it is nice to have it displayed on the screen. Also need to make sure user selects only image files not some other files like executables or scripts which will compromise our web application from malicious activities. We will touch up on the security aspects of the website later. Once we have our mental model, it is time to go drawing board and draw it. 

![](images/skeleton.jpg "skeleton ui")

Let us build our mental model in to actual application and bring it to life.

We did a `git clone` of our branch in inital setup stage. Below tree diagram shows how our code is organized. It looks very daunting at first, don't worry we will break it down layer by layer as we make progress.


```sh
Application
.
├── config.py
├── Dockerfile
├── flask_pytorch_web_app
│   ├── __init__.py
│   ├── model.py
│   ├── routes.py
│   ├── static
│   │   ├── css
│   │   │   └── main.css
│   │   ├── data
│   │   │   └── imagenet_class_index.json
│   │   └── images
│   │       └── opencv-logo-1.png
│   └── templates
│       ├── base.html
│       ├── predict.html
│       └── upload.html
├── local_run.sh
├── README.md
├── requirements.txt
├── setup.py
├── torchapp.ini
└── wsgi.py

```
<br>
First we just need to focus on the base skeleton of the project i.e the html pages.

All of the html and styling aspects of the page lies in the "templates and css" directories.
```sh
│   ├── static
│   │   ├── css
│   │   │   └── main.css
│   ├── templates
│   │   ├── base.html
│   │   ├── predict.html
│   │   └── upload.html
```

Let us have a closer look at the `html` files.

### base.html
Here is the complete `base.html`. 

```html
<!DOCTYPE html>
<html lang='en'>
<head>
  {% block head %} {% endblock %}
</head>

<body>
  <div id="header"> 
    <h3> Flask Web App</h3>
  </div>
  <div id="navbar">  
    <ul>
      <li ><a href="{{ url_for('upload') }}">Home</a></li>
    </ul>
  </div>
  <div class="main">
    <div id="container">
      {% block body%}  {% endblock %}
    </div>
  </div>
  <div id="footer"> 
    <img id="logo" src={{ url_for('static', filename='images/opencv-logo-1.png') }} >
  </div>
</body>

</html>
```

The above html code looks complicated at first. But it is extremely simple to understand, if you know few basics. 
<br>
Flask uses a templating syntax called [Jinja2](https://jinja.palletsprojects.com/en/2.11.x/templates/#) to render the html. `Jinja2` will greatly simplify `html` page rendering through python code. In its simplest form, our  `base.html` looks as shown below. 

```html
<!DOCTYPE html>
<html lang='en'>
<head>
  {% block head %} {% endblock %}
</head>

<body>
  {% block body%} {% endblock %}
</body>

</html>
```

Suppose our web application supports multiple pages. We want to have a similar look and feel for all the pages. The content in the body or header may change. But the over all look and feel should remain same. If we try to hand code all of these elements by hand, it becomes a more and more difficult as we keep adding new features and elements in to our website. It soon becomes a maintenance nightmare to update the website. This is where `Jinja` template parser comes to our rescue. It greatly reduces the effort of creating and maintaining our `html` pages.
<br>

Here `{% block head %} {% endblock %}` and `{% block body%} {% endblock %}` provides a place holder to add or delete  or hide the existing `html tags`. we can use to fill in the required content in new page. `Jinja` templating engine will identifies these blocks and generates the complete html file just when the page is rendered. 
<br>
To understand it even better let us have a look at the `upload.html` file

### upload.html


```html
{% extends 'base.html' %}

{% block head %} 
  <title> Upload Image</title>
{% endblock %}

{% block body %} 
      <div id="form">
        <form method=POST enctype=multipart/form-data action="{{ url_for('upload') }}">
          <label for="inputfile"> Input image file:  </label>
          <input type="file" name="image" id="inputfile" accept=".jpg, .jpeg, .png"> 
          <input type="submit" value="Upload">
        </form>
      </div> 
      {% if obj.is_image_display %}
      <div > 
        <img id="inputImage" src={{ url_for('send_file', filename=obj.image) }} alt="User Image">
      </div>
      {% endif %}
{% endblock %}
```

Here in `upload.html` file we are not writing the complete `html` file, at first line we are just inheriting the `base.html` file using `{% extends 'base.html' %}` since our `base.html` file already has the needed fields.
All you need to do is add or extend the html page according to your needs. `jinja` template engine will do the fill in the blanks for you.



Now in our `update.html` file
```html
{% block head %} 
  <title> Upload Image</title>
{% endblock %}
```
code simply extends the placeholder which we left in `base.html` file in the `head` tag. In this example we are adding a new `title` tag in to our `upload.html` page.

similarly we want an "upload button" which should prompt the user to select an image file upon clicking the same. That is exactly what we are doing with the below piece of html code. This piece of UI we want it to appear in the body portion of the html file. Hence we extend the `{% block body%} {% endblock %}` by putting the required html tags to populate the upload button

```html
{% block body%} 
     <div id="form">
        <form method=POST enctype=multipart/form-data action="{{ url_for('upload') }}">
          <label for="inputfile"> Input image file:  </label>
          <input type="file" name="image" id="inputfile" accept=".jpg, .jpeg, .png"> 
          <input type="submit" value="Upload">
        </form>
      </div> 
{% endblock %}
```

To understand the code in between the tag `<form> ..... </form>` tag it is better to look at the final output and compare it back with our code. Here is our page with some css magic, which we will cover shortly.

![](images/upload_button_only.png)

It is html standard to get the user provided data using `form` tags.

```html 
<form method=POST enctype=multipart/form-data action="{{ url_for('upload') }}">
```

We specify the type of `method` (GET/POST/DELETE/PUT etc) to be used using `method=POST` attribute. In our app we are trying to get the image to be uploaded, hence we need to use `enctype=multipart/form-data` attribute to receive the uploaded image data. `action` attribute is used to pass the image available in the form to a `upload` function in our flask application. The syntax `{{ }}` is the way to tell the `jinja` template engine to find the appropriate method named `upload` in the flask python files. `url_for` is a utility function available in flask library to access the relative path/resources of the flask app. As a best practice, we should always use `url_for` when ever we try to access the resources in our web app, this will help us bind our resources such as logos/css files/images/redirection html files etc to the web app in a secure way.

```html
    <label for="inputfile"> Input image file:  </label>
```
As the tag name `label` suggest it gives a label to our input field to hint the user.

```html
    <input type="file" name="image" id="inputfile" accept=".jpg, .jpeg, .png"> 
```
`input` tag attributes are prettymuch self explainatory. The `accept` attributes tells the browser to filter out and display only the files with extensions `.jpg, .jpeg, .png` to be displayed for selection. This will solve the problem of user accidentally selecting the incorrect types of file such as `.txt or .sh` etc.

```html
    <input type="submit" value="Upload">
```
The attribute `submit` will send the message when user clicks the button using the appropriate methods mentioned in `<form method=POST>`. In our case it is `POST` method.
<br>
`value` attribute is the name which is displayed on the `button`.

### predict.html

Similar to `upload.html` we have `predict.html` file. We use this file when we want to do the prediction of the image. Let us look at the `predict.html` file

```html
{% extends 'base.html' %}

{% block head %} 
  <title> Predict Image</title>
{% endblock %}

{% block body %} 
      {% if obj.is_image_display %}
        {% if obj.is_predicted %}
          <div id="prediction"> 
            <h1> Image has : {{ obj.value }}!! </h1> 
          </div>
        {% else %}
          <div>
            <form method=POST enctype=multipart/form-data action="{{ url_for('prediction', filename=obj.image) }}">
              <input type="submit" value="Classify Image" id="blockButton">
            </form>
          </div> 
        {% endif %}
        <div > 
          <img id="inputImage" src={{ url_for('send_file', filename=obj.image) }} alt="User Image">
        </div>
      {% endif %}
{% endblock %}

```

If you look carefully this file looks very similar to `upload.html` we are still using the same inheritance  `{% extends 'base.html' %}` and extending the `base.html` file and fill in the `{% block body %}` and `{% block body %}`. The additional `{% if %} {% else %}  {% endif %}` are standard `jinja` conditionals to enable or disable `div` elements depending on if the image file exists or not. You can read more about the [jinja condtionals syntax here](https://jinja.palletsprojects.com/en/2.11.x/templates/#if).

### CSS

To beautify the web pages we have written `css` file called `main.css`. You can get very creative with CSS with the look and feel of your web page. There are many tutorials available online to learn and master `css`.
<br>
Instead of teaching everything about css, here we just teach you one trick that will help you understand the `css` file and know which specific element of `html` tag and `css` styling are linked and associated.
<br>
If you find attribute as `id` or `class` in html files, you can search for those names in `css` file to find the style applied on that `div`.

For example in our `upload.html` file:

```html
      <div id="form">
        <form method=POST enctype=multipart/form-data action="{{ url_for('upload') }}">
          <label for="inputfile"> Input image file:  </label>
          <input type="file" name="image" id="inputfile" accept=".jpg, .jpeg, .png"> 
          <input type="submit" value="Upload">
        </form>
      </div> 
```

if you search for "form" in our `main.css` file you should find the word `form` in the `main.css` file.

```css
#form {
  margin: 10px;
  display: block;
  margin-left: auto;
  margin-right: auto;
  width: 50%;
  color: white;
  border: none;
  background-color: #CC7E33;
  padding: 14px 28px;
  font-size: 120%;
  font-weight: bolder;
  cursor: pointer;
  text-align: center;
  border-radius: 5px;
}
```

There you go, this is the `css` style which we have used to in this application for the `form` tag. The "`#`" sign in front of "form" tells us that it is an "`id`" attribute (`<div id="form">`) and if there is a "`.`" it means it is a `class` attribute (`<div class="form">`). To follow along with our simple application, this much information is sufficient for now. You will be able to find the rest of the `css` styling used easily.

We still have not done any linking of page and what action to take on clicking the upload button etc. We will get in to those details shortly. But we have all the UI elements which we wanted. For now we just fast forward a bit to see our results with our initial mental model.
![](images/before_classify_partial.png)

## Making UI Interactive

Now we have all the UI blocks ready to start our plumbing work to make the page interactive. All the plumbing work is handled in the following three python files.

```sh
├── flask_pytorch_web_app
│   ├── __init__.py
│   ├── model.py
│   ├── routes.py
```

### __init__.py

This file provides the abstraction for our Flask application. All we do is create an instance of `app` and return the instance it. This abstraction is important to keep our app modular and maintainable.

```python

"""Initialize app."""
from flask import Flask
from flask_uploads import UploadSet, configure_uploads, IMAGES

photos = UploadSet('photos', IMAGES)

def create_app():
    """Construct the core application."""
    app = Flask(__name__, instance_relative_config=False)
    app.config.from_object('config.Config')

    with app.app_context():
        from . import routes
        configure_uploads(app, photos)
        return app
```

We have the regular flask imports
```python
from flask import Flask
from flask_uploads import UploadSet, configure_uploads, IMAGES
```
One really good thing about Flask application is that, there are so many plugins available. All you need to do is search for those plugins. It will almost always gives you better results than if you had done all of it hand coded. In this example, We are also using a plugin,  called [flask_uploads](https://flask-uploads.readthedocs.io/en/latest/) to handle uploading of the files. `flask_uploads` plugin will simplify the task of uploading files and handling the custom paths to store the files and provides a safe and secure way to access dynamic resources.   

```python
photos = UploadSet('photos', IMAGES)
```
Here `UploadSet` helps us setting the type of upload files we are interested in, In our case we are interested in image files and hence we set the group as `IMAGES`, this is nothing but a list of image file extensions `jpg, jpeg, png, svg, bmp, gif`, we will allow and use in our application to be uploaded.

```python
def create_app()
```
Here we are using the Flask's create_app factory pattern to keep our application flexible and easy to use. This factory pattern will help us segregate our app as development mode or production mode or testing mode. We can also pass in the different configurations which we use for different modes such as debug and production mode. We will talk more about it when we get in to the deployment section.

```python
   app = Flask(__name__, instance_relative_config=False)
   app.config.from_object('config.Config')
```

It is always good practice to override the defalut locations and config paths with the custom paths and custom configurations in your application. We are disabling the custom paths and configs by setting the `instance_relative_config=False` variable as we initialize the flask app. And providing the custom configurations. In our case we have a `Config` class in `config.py` file which we are using to configure the application. We will re-visit the `config` parameter shortly.

```python
    with app.app_context():
        from . import routes
        configure_uploads(app, photos)
        return app
```
The above piece of the code is the real gut which connects the html pages with the python code. 
We are opening the application context with python `with` syntax to handle the resource lifetime. This helps us to close the application gracefully, when errors or exceptions happens in our application, without leaking any resources. 

We configure the `flaks_upload` plugin by calling  `configure_uploads` with the instance which we just created. And finally we return the context of the `app`.

`from . import routes` is the key statement which tells the application where our routes are and the logic of which click of a button should do what action is defined in `routes.py`.


## routes.py

`routes.py` is the file where all the plumbing work happens. Let us have a detailed look in this section.

In our simple application we have just three routes.

```python
    @app.route('/', methods=['GET', 'POST'])
    def upload():
        
    @app.route('/uploads/<filename>')
    def send_file(filename): 

    @app.route('/prediction/<filename>', methods=['GET', 'POST'])
    def prediction(filename): 
 
```

`@app.route('/')` This tells our application to land the user into the home directory or root directory `/` when the user types the `ip/Domain` name of our website. This loads the `upload.html` file in the browser.

`@app.route('/uploads')` route we are using it internally to save the image in the custom path provided during the initial run time of the application.

`@app.route('/prediction/)` route will redirect the user to the prediction page. This loads the `predict.html` file in the browser.

Let us break down each of the routes in detail.

our route to `/` in its simplest form is  as shown below.

```python
@app.route('/', methods=['GET', 'POST'])
def upload():
    return render_template('/upload.html', obj=obj)

```
Just having this line in your code and running the application will land you in to our home page. Which is nothing but a html page with upload button.

![](images/upload_button_only.png)

if you look at our `upload.html` we had the following line.

```html
<form method=POST enctype=multipart/form-data action="{{ url_for('upload') }}">
```

So when you click the `upload` button, a request with `method=POST` as an action `action="{{ url_for('upload') }}` is made. Which is nothing but the `def upload()` function in our `route.py` file.


Also notice that in our `@app.route('/', methods=['GET', 'POST'])`accepts both `['GET', 'POST']` methods. If we don't mention the type of methods, flask will by defualt only accepts `GET` method.
We set the http request method in `<form method=POST ...>` html file. These http methods can be accessed through `request` object. If you notice our imports 

```python
from flask import  request
```
We are importing the flask provided inbuilt `request` API. Which inturn depends on the [requests](https://requests.readthedocs.io/en/master/) library.


Now that we understand how the `GET` and `POST` is filled in the `form` tag of `html` and sent as part of the `request` object, let us now see how to filter the method type.


```python
@app.route('/', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST' and 'image' in request.files:
        print("Hey I got a POST method")
    return render_template('/upload.html', obj=obj)

```

As we can see we are checking if the method type with `if request.method == 'POST'` which we received in the `request` object.

But what is the other check `and 'image' in request.files`  doing here?

To understand that, again we need to look at the `upload.html` file again.

```html
<input type="file" name="image" id="inputfile" accept=".jpg, .jpeg, .png">
```

here `request.files` is set with what ever the value you add with the attribute `name`. In our case we have `name="image"`. The equivalent python code would be `request.files = ["images"]`. Hence we can access this variable associated with the files as `request.files` in our `route.py` code.
 
so `if request.method == 'POST' and 'image' in request.files:` is a check on the Method type and making sure the request is coming from the input field of the `form` we are interested in. Let us see how to access the image file and store it so that we can use the stored image on different pages and use it in our prediction mechanism.


```python
@app.route('/', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST' and 'image' in request.files:
        image_file = request.files['image']
        if checkFileType(image_file.filename):
            filename = photos.save(request.files['image'])
            return render_template('/predict.html', obj=obj)
        return render_template('/upload.html', obj=obj)
    return render_template('/upload.html', obj=obj)
```

`image_file = request.files['image']` we just get the file object. And do a sanity check on the type of the image file uploaded in `checkFileType` and save the file using the `photos` instance which we created using `flask_upload` utility.  Here in our example, we have decided to store the image, because storing the image will simplify our logic of handling the image when doing prediction. Once we are done, we just render the required `html` page using `render_template`.

If we notice carefully we are passing `obj` instance in our `render_template` api along with the html file. 

For `render_template` API we can pass any number of argument to our html template file, `jinja` syntax will render the values of the variables you have passed. For example

```python
 render_template('/hello.html', name="opencv", sometext="Hello OpenCv")
```

```html
<!DOCTYPE html>
<html lang='en'>
<head>
    <title> {{ name }} </title>
</head>

<body>
    <h2> {{ sometext }} </h2>
</body>

</html>
```

This page will load and show "opencv" in the title field of the browser tab and displays the text "Hello OpenCv" in the body of the page. We can use this mechanism to display the dynamic texts or objects in our html page. Similar to `string` we can pass an object and access them in our html page.

```python
class DataObject:
    pass
```

For this purpose we have created `DataObject` class in `routes.py` file and fill in the objects dynamically and pass it to the `render_template` API.

Here is the code with data object.

```python
class DataObject:
    pass

@app.route('/', methods=['GET', 'POST'])
def upload():
    obj = DataObject
    obj.is_image_display = False
    obj.image = ""
    if request.method == 'POST' and 'image' in request.files:
        image_file = request.files['image']
        if checkFileType(image_file.filename):
            filename = photos.save(image_file)
            obj.image = filename
            obj.is_image_display = True
            obj.is_predicted = False
            return render_template('/predict.html', obj=obj)
        return render_template('/upload.html', obj=obj)
    return render_template('/upload.html', obj=obj)

```

As you can see we are setting some values dynamically to `DataObj` 

```python

    obj = DataObject
    obj.is_image_display = False
    obj.image = ""
    if request.method == 'POST' and 'image' in request.files:
        image_file = request.files['image']
        if checkFileType(image_file.filename):
            filename = photos.save(image_file)
            obj.image = filename
            obj.is_image_display = True
            obj.is_predicted = False
```

and passing the object to `render_template('/predict.html', obj=obj)` API.

Now if we go-back and look at the `upload.html` and `predict.html` files we are using these object passed

```html
      {% if obj.is_image_display %}
      <div > 
        <img id="inputImage" src={{ url_for('send_file', filename=obj.image) }} alt="User Image">
      </div>
      {% endif %}
```

`if obj.is_image_display` and `obj.image` will get the dynamic values passed from the `routes.py` file.

Now we have to handle the error conditions, which can happen when we receive the file of not supported format or too small or too big image etc.

```python
@app.route('/', methods=['GET', 'POST'])
def upload():
    obj = DataObject
    obj.is_image_display = False
    obj.image = ""
    if request.method == 'POST' and 'image' in request.files:
        image_file = request.files['image']
        if checkFileType(image_file.filename):
            filename = photos.save(image_file)
            obj.image = filename
            obj.is_image_display = True
            obj.is_predicted = False
            return render_template('/predict.html', obj=obj)
        else:
            if image_file.filename:
                msg = f"{image_file.filename} is not an image file"
            else:
                msg = "Please select an image file"
            flash(msg)
        return render_template('/upload.html', obj=obj)
```

As we can see we are doing error check on the file type alone for now, we can extend it to limit the size of the file etc. We use the `flask`'s built in `flash` error handling methods to inform the user about incorrect inputs.

This flash error messages are handled in `base.html` file

```html
      {% with messages = get_flashed_messages() %}
        {% if messages %}
            {% for message in messages %}
                <!-- Do something with the message -->
                <div id="alert">
                    <span class="closebtn" onclick="this.parentElement.style.display='none';">&times;</span>
                    <strong>ERROR: </strong> {{ message }} 
                </div>

            {% endfor %}
        {% endif %}
      {% endwith %}
```
Again we are using `jinja` syntax's to show it only when there is an error from user inputs. Here is an example error case, when user just hits upload button without selecting any images.

![](images/error.png)

Once we pass all the checks if the file uploading is success then we will be routed to `predict.html` page. In case of error, we will be routed back to our home page.

## predict.py

This is how our page looks, an image with a button.

![](images/before_classify_partial.png)


Now, that we know, how the button clicking actions work, there is nothing much to explain in the code for the prediction button. It is similar to the "Upload Button" explanation  but calls a different function. For the sake of completeness we just look at where the trigger happens.

```html
{% block body %} 
      {% if obj.is_image_display %}
        {% if obj.is_predicted %}
          <div id="prediction"> 
            <h1> Image has : {{ obj.value }}!! </h1> 
          </div>
        {% else %}
          <div>
            <form method=POST enctype=multipart/form-data action="{{ url_for('prediction', filename=obj.image) }}">
              <input type="submit" value="Classify Image" id="blockButton">
            </form>
          </div> 
        {% endif %}
        <div > 
          <img id="inputImage" src={{ url_for('send_file', filename=obj.image) }} alt="User Image">
        </div>
      {% endif %}
{% endblock %}
```

**The important thing to notice here is**

```html
 <form method=POST enctype=multipart/form-data action="{{ url_for('prediction', filename=obj.image) }}">
```

**Which calls the `prediction` function in our `route.py` file**

```python

@app.route('/prediction/<filename>', methods=['GET', 'POST'])
def prediction(filename):
    obj = DataObject
    obj.is_image_display = False
    obj.is_predicted = False
    if request.method == 'POST' and filename:
        val = app.config['UPLOADED_PHOTOS_DEST']+filename
        obj.image = filename
        jf = url_for('static', filename='data/imagenet_class_index.json')
        p = predict(val, jf)
        if len(p) == 2:
            obj.is_image_display = True
            obj.is_predicted = True
            obj.value = cleanString(p[1])
            return render_template('/predict.html', obj=obj)
        else:
            flash(f'Something went wrong with prediction. Try a different image')
            return render_template('/predict.html', obj=obj)
    return render_template('/upload.html', obj=obj)

```

Most of the code looks similar to what we have already seen. But only couple of things we need to pay special attention to. i.e

```python
        jf = url_for('static', filename='data/imagenet_class_index.json')
```
In this example we are keeping a json file under `data` directory, we are using the `url_for` flask API to pass the file our prectict API
```python
        p = predict(val, jf)
```

This is where we are calling the pytorch models to predict the user uploaded image. Our `predict` function takes 2 input arguments. First is the location of the user uploaded image file where we have stored and the json file which has the classes defined. 

Here we are keeping our model related code isolated from the python web app UI related code.

## Build the model

### model.py

This is where we are using the built model.

```python

def predict(image_file, class_file):
    class_file = getcwd() + '/flask_pytorch_web_app/' + class_file
    # Make sure to pass `pretrained` as `True` to use the pretrained weights:
    model = models.densenet121(pretrained=True)
    # Since we are using our model only for inference, switch to `eval` mode:
    model.eval()

    imagenet_class_index = json.load(open(class_file))
    with open(image_file, 'rb') as f:
        image_bytes = f.read()
        tensor = transform_image(image_bytes=image_bytes)
        outputs = model.forward(tensor)
        _, y_hat = outputs.max(1)
        predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]

```

In this exercise we will be using the pytorch's builtin `densenet121` model to classify the image.

```python
    model = models.densenet121(pretrained=True)
    model.eval()
```
We need to put the model in `model.eval()` mode. As we are doing the evaluation on the set of user provided images.

```python
    imagenet_class_index = json.load(open(class_file))
    with open(image_file, 'rb') as f:
        image_bytes = f.read()
        tensor = transform_image(image_bytes=image_bytes)
        outputs = model.forward(tensor)
        _, y_hat = outputs.max(1)
        predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]
```

We load the json file and we transform the user provided image using the `transform_image`. This is required because, the `densenet121` model is trained on an image of size `255*255`


```python
def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)
```
Along with re-sizing the image, we also do center crop, convert it in to a tensor and normalize. We use pytorch's `transforms.Compose` to do the same. And return the image as tensor.



The output of the `transform_image` function is fed in to our model and we pick the maximum probable value index from the output.

```python
        outputs = model.forward(tensor)
        _, y_hat = outputs.max(1)
        predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]
```

Here is our sample json file.

```json
{"0": ["n01440764", "tench"],
 "1": ["n01443537", "goldfish"],
 "2": ["n01484850", "great_white_shark"],
 ....
}
```
Our model returns an number ranging between 0 and 999. We type cast this number to string and use this as our key in getting the data from the `imagenet_class_index.json` file. 

The returned value is processed in our `routes.py` file

```python
# routes.py
        p = predict(val, jf)
        if len(p) == 2:
            obj.is_image_display = True
            obj.is_predicted = True
            obj.value = cleanString(p[1])
            return render_template('/predict.html', obj=obj)
```
This will pick only the actual name and render our page `predict.html`. That will finally show us the result in the expected format as shown below.

![](images/final_output.png)


### Local build and test

You can run the application locally before going in to deployment.

Let us have a separate virtual environment for our development.

```sh
$ sudo apt install python3-venv
$ python3 -m venv webappenv
$ source ./webapp/bin/activate

$ git clone https://github.com/bigvisionai/pytorch-web-app-deploy-azure
$ cd pytorch-web-app-deploy-azure
$ pip install -r requirements.txt

```
Let us run the app locally.

```sh
$ bash local_run.sh
[2020-05-14 12:50:51 +0200] [19110] [INFO] Starting gunicorn 20.0.4
[2020-05-14 12:50:51 +0200] [19110] [INFO] Listening at: http://0.0.0.0:8000 (19110)
[2020-05-14 12:50:51 +0200] [19110] [INFO] Using worker: sync
[2020-05-14 12:50:51 +0200] [19113] [INFO] Booting worker with pid: 19113
[2020-05-14 12:50:51 +0200] [19114] [INFO] Booting worker with pid: 19114
[2020-05-14 12:50:51 +0200] [19115] [INFO] Booting worker with pid: 19115
[2020-05-14 12:50:51 +0200] [19116] [INFO] Booting worker with pid: 19116
```
Now navigate to the browser and open the link `http://0.0.0.0:8000`.

We are all set to deploy our application! Let us see the options we have to deploy in our next section.
